In [1]:
import cv2 
import numpy as np

def text_to_bin(text):
    return ''.join(format(ord(x), '08b') if x != ' ' else '00100000' for x in text)

def bin_to_text(bin_str):
    chars = []
    for i in range(0, len(bin_str), 8):
        byte = bin_str[i:i+8]
        chars.append(chr(int(byte, 2)))
    return ''.join(chars)

def bin_to_num(bin_num):
    exp = 7
    num = 0
    for bit in bin_num:
        if bit == '1':
            num += (2 ** exp)
        exp -= 1
    return num

def num_to_bin(num):
    bin_num = ""
    while True:
        bin_num += str(num % 2)
        num = int(num / 2)
        if num == 0:
            break
    len_num = len(bin_num)
    for i in range(8 - len_num):
        bin_num += "0"
    return bin_num[::-1]
    

def encode(image,secret,key):
    flat_images = image.flatten()
    flat_images_bin = []
    stego = []
    
    for num in flat_images:
        flat_images_bin.append(num_to_bin(num))
        
    index = 0
    
    for pixel in flat_images_bin:
        new_text = ""
        
                
        if(index < key):
            for i in range(len(pixel)):
                if(i == 6):
                    new_text += secret[index]
                    index += 1
                elif(i == 7):
                    if(index < key):
                        new_text += secret[index]
                        index += 1
                    else:
                        new_text += pixel[i]
                else:
                    new_text += pixel[i]
            stego.append(new_text)
        else:
            stego.append(pixel)
    
    stego_img = []
    for each in stego:
        stego_img.append(bin_to_num(each))
        
    return stego_img

def decode(stego,key):
    flat_stegos = stego.flatten()
    flat_stegos_bin = []
    secret_bin = ""
    
    for stego in flat_stegos:
        flat_stegos_bin.append(num_to_bin(stego))
    
    index = 0
    for pixel in flat_stegos_bin:
        secret_bin += pixel[6]
        index += 1
        if(index > key-1): break
        
        secret_bin += pixel[7]
        index += 1
        if(index > key-1): break
    
    return bin_to_text(secret_bin)

In [ ]:
img = cv2.imread("00002.jpg")
secret_text = "This is the most secure SENTENCE that no one can touch haha"
secret_text_bin = text_to_bin(secret_text)
print(secret_text_bin)
key = len(secret_text_bin)

stego = np.array(encode(img,secret_text_bin,key))
stego_img = np.reshape(stego, img.shape)
# Convert stego image to uint8 type
stego_img = stego_img.astype(np.uint8)

cv2.imshow("Cover Image", img)
cv2.imshow("Stego Image", stego_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

0101010001101000011010010111001100100000011010010111001100100000011101000110100001100101001000000110110101101111011100110111010000100000011100110110010101100011011101010111001001100101001000000101001101000101010011100101010001000101010011100100001101000101001000000111010001101000011000010111010000100000011011100110111100100000011011110110111001100101001000000110001101100001011011100010000001110100011011110111010101100011011010000010000001101000011000010110100001100001


In [ ]:
decode(stego_img,key)

'This is the most secure SENTENCE that no one can touch haha'